In [1]:
import pandas as pd
import os
from pathlib import Path
import tokenisation as tkn
import api_calls as api
import running_model as rnm
import json
import regex as re

In [2]:
WRITE_DIR = Path("./data/results/prompt_1/gpt-3.5-turbo/speeches_2/")

In [3]:
with open("./data/prompts/prompt_1.txt", "r") as file:
    prompt_1 = file.read().splitlines()

In [7]:
data = pd.read_parquet("./data/scraping/all_fed_speeches_2020_to_2023.parquet")

In [9]:
# title_words = ["policy", "economy", "outlook", "economic", "brief", "summary", "monetary", "future", "economy", "unemployment", "inflation", "update"]
title_words = ["policy", "outlook", "economic", "brief", "summary", "monetary", "future", "economy", "unemployment", "inflation", "update"]

In [14]:
count = 0
for i in range(data.shape[0]):
    if not any(ele in data.iloc[i, 2].lower() for ele in title_words):
        continue
    title = re.search(r'.*\/([^\/]+)\.[^\.]+$', data.iloc[i,3]).group(1)
    if title == "kroszner2007116a":
        continue
    if os.path.exists(f"./data/results/prompt_1/gpt-3.5-turbo/speeches_2/gpt-3.5-turbo_prompt_1_{title}.json"):
        print(f"done {title}, moving on...")
        continue
    print(data.iloc[i,0], data.iloc[i, 2])
    result = []
    sentences = tkn.get_important_text(tkn.basic_tokeniser(data.iloc[i, -1]))
    raw_sentiments = await api.get_multiple_api_calls("gpt-3.5-turbo", prompt_1, sentences)
    labels = [rnm._find_label(sentiment[1]) for sentiment in raw_sentiments]
    reasons = [sentiment[0] for sentiment in raw_sentiments]
    for sentence, label, reason in zip(sentences, labels, reasons):
        result.append({"sentence": sentence, "label": label, "reason" : reason})
    with open(f"./data/results/prompt_1/gpt-3.5-turbo/speeches_2/gpt-3.5-turbo_prompt_1_{title}.json", "w") as file:
        json.dump(result, file)


done brainard20201117a, moving on...
done clarida20201019a, moving on...
done clarida20201014a, moving on...
done powell20201006a, moving on...
done brainard20200901a, moving on...
done clarida20200831a, moving on...
done powell20200827a, moving on...
done bowman20200826a, moving on...
done bowman20200819a, moving on...
done brainard20200813a, moving on...
done brainard20200806a, moving on...
done brainard20200714a, moving on...
done clarida20200616a, moving on...
done clarida20200521a, moving on...
done powell20200513a, moving on...
done powell20200409a, moving on...
done clarida20200225a, moving on...
done clarida20200221a, moving on...
done brainard20200221a, moving on...
done quarles20200206a, moving on...
done bowman20200116a, moving on...
done clarida20200109a, moving on...
done waller20211217a, moving on...
done bowman20211129a, moving on...
done clarida20211119a, moving on...
done waller20211119a, moving on...
done clarida20211108a, moving on...
done quarles20211020a, moving on

In [12]:
data

,date,speaker,title,link,text
0,2020-12-18,Governor Lael Brainard,Strengthening the Financial System to Meet the...,/newsevents/speech/brainard20201218a.htm,I want to thank the Center for American Progre...
1,2020-12-17,Governor Lael Brainard,Modernizing and Strengthening CRA Regulations:...,/newsevents/speech/brainard20201217a.htm,Good afternoon and thank you for inviting me t...
2,2020-12-11,Vice Chair for Supervision Randal K. Quarles,The Eye of Providence: Thoughts on the Evoluti...,/newsevents/speech/quarles20201211a.htm,Torrential thanks to our partners Harvard Law ...
3,2020-12-04,Governor Michelle W. Bowman,Technology and the Regulatory Agenda for Commu...,/newsevents/speech/bowman20201204a.htm,Thank you to the Independent Community Bankers...
4,2020-12-01,Governor Lael Brainard,Modernizing and Strengthening CRA Regulations:...,/newsevents/speech/brainard20201201a.htm,Good morning. I want to thank Dr. Helene Gayle...
...,...,...,...,...,...
229,2023-01-20,Governor Christopher J. Waller,A Case for Cautious Optimism,/newsevents/speech/waller20230120a.htm,"Thank you, Ben, and thank you to the Council o..."
230,2023-01-19,Vice Chair Lael Brainard,Staying the Course to Bring Inflation Down,/newsevents/speech/brainard20230119a.htm,"Inflation has declined in recent months, which..."
231,2023-01-10,Governor Michelle W. Bowman,Brief Remarks on the Economy and Bank Supervision,/newsevents/speech/bowman20230110a.htm,Governor Bowman presented identical remarks to...
232,2023-01-10,Chair Jerome H. Powell,"Panel on ""Central Bank Independence and the Ma...",/newsevents/speech/powell20230110a.htm,"I will address three main points. First, the F..."


In [28]:
any(ele in data.iloc[51, 2] for ele in title_words)

False

In [31]:
count

294

In [9]:
data["title"].to_csv("quick.csv")

In [24]:
data.iloc[51]

date                                     2006-04-13 00:00:00
speaker                               Governor Mark W. Olson
title                             Update on the U.S. economy
link                   /newsevents/speech/olson20060413a.htm
text       Recent trends in the U.S. economy have been, o...
Name: 51, dtype: object